In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format
#from pandas.plotting import autocorrelation_plot
from math import sqrt
#import researchpy as rp
import scipy as sp
from scipy import stats
import scipy.signal
from scipy.signal import fftconvolve, convolve
import matplotlib.pyplot as plt
import seaborn as sns
#from ipywidgets import interact
import random as ran
from pprint import pprint

from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import *

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn import datasets
#from textblob import TextBlob, Word
#from nltk.stem.snowball import SnowballStemmer
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from nltk.corpus import stopwords
#from textacy.preprocess import preprocess_text

#from gensim import corpora, models, matutils
from collections import defaultdict, Counter

import statsmodels.api as sm
import statsmodels.formula.api as sm
from statsmodels.tsa.stattools import acf, pacf, adfuller
#from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.statespace.sarimax import SARIMAX

#import tensorflow as tf

import patsy
from itertools import combinations

#from imblearn.over_sampling import SMOTE

#import graphviz
import json
import requests
import urllib
#from bs4 import BeautifulSoup
#from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from time import sleep
import re
#import twitter

#%matplotlib inline
#plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [2]:
now = datetime.now()
print('Current date and time:')
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time:
2019-08-15 23:53:34

In [3]:
spark.version

Out[ 170 ]: '2.4.1'

In [4]:
spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion")

Out[ 171 ]: '5.3.x-scala2.11'

## Build and document a working model, prototype, recommendation, or solution

The [Chicago Crime Data](https://console.cloud.google.com/marketplace/details/city-of-chicago-public-data/chicago-crime?filter=solution-type:dataset&id=a985ccaf-0a3a-4eb9-a2de-c4fd07de08f0)set is hosted in Google BigQuery. I retrieved this BigQuery data as a Pandas DataFrame and then exported the DataFrame as a CSV file which will be used in the data modeling.

### Chicago Crime Data
<img src="https://upload.wikimedia.org/wikipedia/commons/5/5e/Seal_of_Chicago%2C_Illinois.png" style="float: center; margin: 20px; height: 100px">
City of Chicago <br>
Chicago Police Department crime data from 2001 to present

#### Overview

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified.

This data includes unverified reports supplied to the Police Department. The preliminary crime classifications may be changed at a later date based upon additional investigation and there is always the possibility of mechanical or human error. Therefore, the Chicago Police Department does not guarantee (either expressed or implied) the accuracy, completeness, timeliness, or correct sequencing of the information. The information should not be used for comparison purposes over time.

<table style="width:100%;">
<colgroup>
<col width="15%" />
<col width="10%" />
<col width="10%" />
<col width="65%" />
</colgroup>
<thead>
<tr class="header">
<th align="left">Field name</th>
<th align="left">Type</th>
<th align="left">Mode</th>
<th align="left">Description</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td align="left">unique_key</td>
<td align="right">INTEGER</td>
<td align="left">REQUIRED</td>
<td align="left">Unique identifier for the record.</td>
</tr>
<tr class="even">
<td align="left">case_number</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.</td>
</tr>
<tr class="odd">
<td align="left">date</td>
<td align="left">TIMESTAMP</td>
<td align="left">NULLABLE</td>
<td align="left">Date when the incident occurred. this is sometimes a best estimate.</td>
</tr> 
<tr class="even">
<td align="left">block</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The partially redacted address where the incident occurred, placing it on the same block as the actual address.</td>
</tr> 
<tr class="odd">
<td align="left">iucr</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.</td>
</tr> 
<tr class="even">
<td align="left">primary_type</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The primary description of the IUCR code.</td>
</tr>
<tr class="odd">
<td align="left">description</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The secondary description of the IUCR code, a subcategory of the primary description.</td>
</tr> 
<tr class="even">
<td align="left">location_description</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">Description of the location where the incident occurred.</td>
</tr>
<tr class="odd">
<td align="left">arrest</td>
<td align="left">BOOLEAN</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates whether an arrest was made.</td>
</tr> 
<tr class="even">
<td align="left">domestic</td>
<td align="left">BOOLEAN</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.</td>
</tr>
<tr class="odd">
<td align="left">beat</td>
<td align="left">INTEGER</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.</td>
</tr> 
<tr class="even">
<td align="left">district</td>
<td align="left">INTEGER</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.</td>
</tr>
<tr class="odd">
<td align="left">ward</td>
<td align="left">INTEGER</td>
<td align="left">NULLABLE</td>
<td align="left">The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.</td>
</tr> 
<tr class="even">
<td align="left">community_area</td>
<td align="left">INTEGER</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.</td>
</tr>
<tr class="odd">
<td align="left">fbi_code</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.</td>
</tr> 
<tr class="even">
<td align="left">x_coordinate</td>
<td align="left">FLOAT</td>
<td align="left">NULLABLE</td>
<td align="left">The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.</td>
</tr>
<tr class="odd">
<td align="left">y_coordinate</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.</td>
</tr> 
<tr class="even">
<td align="left">year</td>
<td align="left">INTEGER</td>
<td align="left">NULLABLE</td>
<td align="left">Year the incident occurred.</td>
</tr>
<tr class="odd">
<td align="left">updated_on</td>
<td align="left">TIMESTAMP</td>
<td align="left">NULLABLE</td>
<td align="left">Date and time the record was last updated.</td>
</tr> 
<tr class="even">
<td align="left">latitude</td>
<td align="left">FLOAT</td>
<td align="left">NULLABLE</td>
<td align="left">The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.</td>
</tr>
<tr class="odd">
<td align="left">longitude</td>
<td align="left">FLOAT</td>
<td align="left">NULLABLE</td>
<td align="left">The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.</td>
</tr>
<tr class="even">
<td align="left">location</td>
<td align="left">STRING</td>
<td align="left">NULLABLE</td>
<td align="left">The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.</td>
</tr>
</tbody>
</table>

### Reading a File

In [11]:
display(dbutils.fs.ls('dbfs:/FileStore/tables/'))

path,name,size
dbfs:/FileStore/tables/chicago_crime.csv,chicago_crime.csv,1702985041
dbfs:/FileStore/tables/hour.csv,hour.csv,1156736


In [12]:
# Create file location and type for loading data
file_location = '/FileStore/tables/chicago_crime.csv'
file_type = 'csv'

# CSV options
infer_schema = 'true'
first_row_is_header = 'true'
delimiter = ','

# Read file into DataFrame
# The applied options are for CSV files. For other file types, these will be ignored.
chicago_crime = (spark.read.format(file_type)
  .option('inferSchema', infer_schema)
  .option('header', first_row_is_header)
  .option('sep', delimiter)
  .load(file_location))

display(chicago_crime)

unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
11691617,JC264900,2019-05-15 19:00:00+00:00,095XX S AVALON AVE,1750,OFFENSE INVOLVING CHILDREN,CHILD ABUSE,RESIDENCE,false,true,511,5.0,8.0,50.0,08B,1186206.0,1841979.0,2019,2019-05-22 16:12:53+00:00,41.721514573,-87.593555694,"(41.721514573, -87.593555694)"
11679177,JC252697,2019-05-06 20:09:00+00:00,052XX S WELLS ST,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,RESIDENCE,false,false,935,9.0,3.0,37.0,26,1175491.0,1870242.0,2019,2019-05-13 16:13:25+00:00,41.79931753,-87.631958462,"(41.79931753, -87.631958462)"
11673919,JC246944,2019-05-02 15:22:00+00:00,033XX W FOSTER AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,SIDEWALK,true,false,1712,17.0,39.0,13.0,24,1152975.0,1934347.0,2019,2019-05-09 16:13:26+00:00,41.975702619,-87.71283079,"(41.975702619, -87.71283079)"
11638281,JC203800,2019-03-29 08:45:00+00:00,087XX S STONY ISLAND AVE,0497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,PARKING LOT/GARAGE(NON.RESID.),false,true,412,4.0,8.0,48.0,04B,1188323.0,1847328.0,2019,2019-04-05 16:07:16+00:00,41.736142605,-87.585631434,"(41.736142605, -87.585631434)"
24442,JC195241,2019-03-22 11:55:00+00:00,101XX S AVENUE L,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,false,false,432,4.0,10.0,52.0,01A,1201819.0,1838463.0,2019,2019-03-29 16:12:07+00:00,41.711483998,-87.536488614,"(41.711483998, -87.536488614)"
11610775,JC170595,2019-03-01 21:00:00+00:00,130XX S EVANS AVE,0326,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,VEHICLE NON-COMMERCIAL,false,false,533,5.0,9.0,54.0,03,1183713.0,1818812.0,2019,2019-03-08 16:36:43+00:00,41.657999653,-87.603406021,"(41.657999653, -87.603406021)"
11591492,JC147084,2019-02-10 11:21:00+00:00,038XX E 112TH ST,502R,OTHER OFFENSE,VEHICLE TITLE/REG OFFENSE,STREET,false,false,433,4.0,10.0,52.0,26,1203282.0,1831314.0,2019,2019-02-17 16:05:09+00:00,41.691829279,-87.531374687,"(41.691829279, -87.531374687)"
11604634,JC163019,2019-02-04 15:00:00+00:00,058XX N MILWAUKEE AVE,1120,DECEPTIVE PRACTICE,FORGERY,BANK,false,false,1622,16.0,45.0,10.0,10,1135151.0,1938417.0,2019,2019-02-27 16:12:30+00:00,41.987206625,-87.778280142,"(41.987206625, -87.778280142)"
11582324,JC135389,2019-01-31 14:02:00+00:00,062XX S HARLEM AVE,0520,ASSAULT,AGGRAVATED:KNIFE/CUTTING INSTR,OTHER,false,false,812,8.0,23.0,64.0,04A,1129454.0,1862389.0,2019,2019-02-07 16:13:02+00:00,41.778674144,-87.800968982,"(41.778674144, -87.800968982)"
11570528,JC121239,2019-01-18 09:20:00+00:00,056XX S KEELER AVE,0520,ASSAULT,AGGRAVATED:KNIFE/CUTTING INSTR,RESIDENCE,false,true,813,8.0,13.0,62.0,04A,1149334.0,1866666.0,2019,2019-01-25 16:14:00+00:00,41.790050062,-87.72797583,"(41.790050062, -87.72797583)"


### Examine the DataFrame

In [14]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6876354, Cols: 22

In [15]:
# Prints out the schema in the tree format
chicago_crime.printSchema()

root
-- unique_key: integer (nullable = true)
-- case_number: string (nullable = true)
-- date: string (nullable = true)
-- block: string (nullable = true)
-- iucr: string (nullable = true)
-- primary_type: string (nullable = true)
-- description: string (nullable = true)
-- location_description: string (nullable = true)
-- arrest: boolean (nullable = true)
-- domestic: boolean (nullable = true)
-- beat: integer (nullable = true)
-- district: double (nullable = true)
-- ward: double (nullable = true)
-- community_area: double (nullable = true)
-- fbi_code: string (nullable = true)
-- x_coordinate: double (nullable = true)
-- y_coordinate: double (nullable = true)
-- year: integer (nullable = true)
-- updated_on: string (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- location: string (nullable = true)

In [16]:
# Returns all column names as a list
chicago_crime.columns

Out[ 176 ]: 
['unique_key',
 'case_number',
 'date',
 'block',
 'iucr',
 'primary_type',
 'description',
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'x_coordinate',
 'y_coordinate',
 'year',
 'updated_on',
 'latitude',
 'longitude',
 'location']

In [17]:
# Returns a new DataFrame sorted by the specified column(s)
# Note: ascending: boolean or list of boolean (default True); sort ascending vs. descending; specify list for multiple sort orders;
#       if a list is specified, length of the list must equal length of the cols
#chicago_crime.sort(['X', 'Y'], ascending=True).collect()

In [18]:
# Prints the first n rows to the console
# Note: truncate: whether truncate long strings and align cells right
chicago_crime.show(5, truncate=False)

+----------+-----------+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
unique_key|case_number|date |block |iucr|primary_type |description |location_description |arrest|domestic|beat|district|ward|community_area|fbi_code|x_coordinate|y_coordinate|year|updated_on |latitude |longitude |location |
+----------+-----------+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
11691617 |JC264900 |2019-05-15 19:00:00+00:00|095XX S AVALON AVE |1750|OFFENSE INVOLVING CHILDREN|CHILD ABUSE |RESIDENCE |false |true |511 |5.0 |8.0 |50.0 |08B |1186206.0 |1841979.0 |2019|2019-05-22 16:12:53+00:00|41.721514573|-87.593555694|(41.721514573, -87.593555694)|
11679177 |JC252697 |2019-05-06 20:09:00+00:00|052XX S WELLS ST |4651|OTHER OFFENSE |SEX OFFENDER: FAIL REG NEW ADD |RESIDENCE |false |false |935 |9.0 |3.0 |37.0 |26 |1175491.0 |1870242.0 |2019|2019-05-13 16:13:25+00:00|41.79931753 |-87.631958462|(41.79931753, -87.631958462) |
11673919 |JC246944 |2019-05-02 15:22:00+00:00|033XX W FOSTER AVE |0470|PUBLIC PEACE VIOLATION |RECKLESS CONDUCT |SIDEWALK |true |false |1712|17.0 |39.0|13.0 |24 |1152975.0 |1934347.0 |2019|2019-05-09 16:13:26+00:00|41.975702619|-87.71283079 |(41.975702619, -87.71283079) |
11638281 |JC203800 |2019-03-29 08:45:00+00:00|087XX S STONY ISLAND AVE|0497|BATTERY |AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON|PARKING LOT/GARAGE(NON.RESID.)|false |true |412 |4.0 |8.0 |48.0 |04B |1188323.0 |1847328.0 |2019|2019-04-05 16:07:16+00:00|41.736142605|-87.585631434|(41.736142605, -87.585631434)|
24442 |JC195241 |2019-03-22 11:55:00+00:00|101XX S AVENUE L |0110|HOMICIDE |FIRST DEGREE MURDER |STREET |false |false |432 |4.0 |10.0|52.0 |01A |1201819.0 |1838463.0 |2019|2019-03-29 16:12:07+00:00|41.711483998|-87.536488614|(41.711483998, -87.536488614)|
+----------+-----------+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
only showing top 5 rows

In [19]:
# Returns the first row as a Row
#chicago_crime.first()

# Returns the first num rows as a list of Row
#chicago_crime.take(5)

#### Identifying important columns to include in the DataFrame.

In [21]:
# Select multiple columns
include_cols = ['date', 'block', 'iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic',
                'beat', 'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate', 'y_coordinate', 'year',
                'updated_on', 'latitude', 'longitude', 'location'] 
chicago_crime = chicago_crime[include_cols]

In [22]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6876354, Cols: 20

In [23]:
# Prints the first n rows to the console
# Note: truncate: whether truncate long strings and align cells right
chicago_crime.show(5, truncate=False)

+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
date |block |iucr|primary_type |description |location_description |arrest|domestic|beat|district|ward|community_area|fbi_code|x_coordinate|y_coordinate|year|updated_on |latitude |longitude |location |
+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
2019-05-15 19:00:00+00:00|095XX S AVALON AVE |1750|OFFENSE INVOLVING CHILDREN|CHILD ABUSE |RESIDENCE |false |true |511 |5.0 |8.0 |50.0 |08B |1186206.0 |1841979.0 |2019|2019-05-22 16:12:53+00:00|41.721514573|-87.593555694|(41.721514573, -87.593555694)|
2019-05-06 20:09:00+00:00|052XX S WELLS ST |4651|OTHER OFFENSE |SEX OFFENDER: FAIL REG NEW ADD |RESIDENCE |false |false |935 |9.0 |3.0 |37.0 |26 |1175491.0 |1870242.0 |2019|2019-05-13 16:13:25+00:00|41.79931753 |-87.631958462|(41.79931753, -87.631958462) |
2019-05-02 15:22:00+00:00|033XX W FOSTER AVE |0470|PUBLIC PEACE VIOLATION |RECKLESS CONDUCT |SIDEWALK |true |false |1712|17.0 |39.0|13.0 |24 |1152975.0 |1934347.0 |2019|2019-05-09 16:13:26+00:00|41.975702619|-87.71283079 |(41.975702619, -87.71283079) |
2019-03-29 08:45:00+00:00|087XX S STONY ISLAND AVE|0497|BATTERY |AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON|PARKING LOT/GARAGE(NON.RESID.)|false |true |412 |4.0 |8.0 |48.0 |04B |1188323.0 |1847328.0 |2019|2019-04-05 16:07:16+00:00|41.736142605|-87.585631434|(41.736142605, -87.585631434)|
2019-03-22 11:55:00+00:00|101XX S AVENUE L |0110|HOMICIDE |FIRST DEGREE MURDER |STREET |false |false |432 |4.0 |10.0|52.0 |01A |1201819.0 |1838463.0 |2019|2019-03-29 16:12:07+00:00|41.711483998|-87.536488614|(41.711483998, -87.536488614)|
+-------------------------+------------------------+----+--------------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------------+------------+-------------+-----------------------------+
only showing top 5 rows

In [24]:
# Returns a new DataFrame by renaming an existing column
chicago_crime = chicago_crime.withColumnRenamed('location', 'lat_long')

In [25]:
# Prints out the schema in the tree format
chicago_crime.printSchema()

root
-- date: string (nullable = true)
-- block: string (nullable = true)
-- iucr: string (nullable = true)
-- primary_type: string (nullable = true)
-- description: string (nullable = true)
-- location_description: string (nullable = true)
-- arrest: boolean (nullable = true)
-- domestic: boolean (nullable = true)
-- beat: integer (nullable = true)
-- district: double (nullable = true)
-- ward: double (nullable = true)
-- community_area: double (nullable = true)
-- fbi_code: string (nullable = true)
-- x_coordinate: double (nullable = true)
-- y_coordinate: double (nullable = true)
-- year: integer (nullable = true)
-- updated_on: string (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- lat_long: string (nullable = true)

### Examine Missing Values

In [27]:
from pyspark.sql.functions import isnan, when, sum, count, col

# Count the missing values in each column of a DataFrame
null_counts = chicago_crime.select(*(sum(col(c).isNull().cast('int')).alias(c) for c in chicago_crime.columns))
null_counts.show()

+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
date|block|iucr|primary_type|description|location_description|arrest|domestic|beat|district| ward|community_area|fbi_code|x_coordinate|y_coordinate|year|updated_on|latitude|longitude|lat_long|
+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
 0| 0| 0| 0| 0| 5013| 0| 0| 0| 47|614824| 613496| 0| 64983| 64983| 0| 0| 64983| 64983| 64983|
+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+

In [28]:
# Count the missing values in each column of a DataFrame
#names = chicago_crime.schema.names
#for name in names:
#    if chicago_crime.where(chicago_crime[name].isNull()).count() > 0:
#        print(str(name), chicago_crime.where(chicago_crime[name].isNull()).count())

In [29]:
# Count the missing values in each column of a DataFrame - sum() works because True is 1 and False is 0
#null_counts = chicago_crime.isNull().sum()
#null_counts[null_counts > 0]

In [30]:
# Create a vertical bar plot by number of missing values per column using matplotlib
# Note: figsize: a tuple (width, height) in inches
#chicago_crime.isnull().sum().plot.bar(figsize=(10, 8));
#plt.title('Number of null values per column');

### Examine the Columns

In [32]:
import pyspark.sql.functions as F
def value_counts(df, colName, show_n=50):
    return (df.groupBy(colName).count().sort('count', ascending=False).withColumn('percent', F.round((F.col('count') / df.count()), 3)).show(show_n, truncate=False))

In [33]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'primary_type')

+---------------------------------+-------+-------+
primary_type |count |percent|
+---------------------------------+-------+-------+
THEFT |1448232|0.211 |
BATTERY |1256079|0.183 |
CRIMINAL DAMAGE |784264 |0.114 |
NARCOTICS |718354 |0.104 |
ASSAULT |428501 |0.062 |
OTHER OFFENSE |427465 |0.062 |
BURGLARY |392857 |0.057 |
MOTOR VEHICLE THEFT |318907 |0.046 |
DECEPTIVE PRACTICE |272813 |0.04 |
ROBBERY |259579 |0.038 |
CRIMINAL TRESPASS |196748 |0.029 |
WEAPONS VIOLATION |73573 |0.011 |
PROSTITUTION |68635 |0.01 |
PUBLIC PEACE VIOLATION |48561 |0.007 |
OFFENSE INVOLVING CHILDREN |46697 |0.007 |
CRIM SEXUAL ASSAULT |28085 |0.004 |
SEX OFFENSE |25749 |0.004 |
INTERFERENCE WITH PUBLIC OFFICER |15801 |0.002 |
GAMBLING |14450 |0.002 |
LIQUOR LAW VIOLATION |14146 |0.002 |
ARSON |11336 |0.002 |
HOMICIDE |9680 |0.001 |
KIDNAPPING |6754 |0.001 |
INTIMIDATION |4028 |0.001 |
STALKING |3490 |0.001 |
OBSCENITY |607 |0.0 |
CONCEALED CARRY LICENSE VIOLATION|374 |0.0 |
NON-CRIMINAL |170 |0.0 |
PUBLIC INDECENCY |165 |0.0 |
OTHER NARCOTIC VIOLATION |126 |0.0 |
HUMAN TRAFFICKING |57 |0.0 |
NON - CRIMINAL |38 |0.0 |
RITUALISM |23 |0.0 |
NON-CRIMINAL (SUBJECT SPECIFIED) |9 |0.0 |
DOMESTIC VIOLENCE |1 |0.0 |
+---------------------------------+-------+-------+

#### Drop non-criminal observations

In [35]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('non_criminal', when(chicago_crime.primary_type.isin(['NON-CRIMINAL', 'NON - CRIMINAL', 'NON-CRIMINAL (SUBJECT SPECIFIED)']), 1).otherwise(0))

In [36]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'non_criminal')

+------------+-------+-------+
non_criminal|count |percent|
+------------+-------+-------+
0 |6876137|1.0 |
1 |217 |0.0 |
+------------+-------+-------+

In [37]:
# Excluding non-criminal values
chicago_crime = chicago_crime[chicago_crime.non_criminal == 0]
chicago_crime = chicago_crime.drop('non_criminal')

In [38]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6876137, Cols: 20

#### Combining 'OTHER NARCOTIC VIOLATION' with 'NARCOTICS'

In [40]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('primary_type', when(chicago_crime.primary_type == 'OTHER NARCOTIC VIOLATION', 'NARCOTICS').otherwise(chicago_crime.primary_type))

#### Combining 'DOMESTIC VIOLENCE' with 'ASSAULT'

In [42]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('primary_type', when(chicago_crime.primary_type == 'DOMESTIC VIOLENCE', 'ASSAULT').otherwise(chicago_crime.primary_type))

In [43]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'primary_type')

+---------------------------------+-------+-------+
primary_type |count |percent|
+---------------------------------+-------+-------+
THEFT |1448232|0.211 |
BATTERY |1256079|0.183 |
CRIMINAL DAMAGE |784264 |0.114 |
NARCOTICS |718480 |0.104 |
ASSAULT |428502 |0.062 |
OTHER OFFENSE |427465 |0.062 |
BURGLARY |392857 |0.057 |
MOTOR VEHICLE THEFT |318907 |0.046 |
DECEPTIVE PRACTICE |272813 |0.04 |
ROBBERY |259579 |0.038 |
CRIMINAL TRESPASS |196748 |0.029 |
WEAPONS VIOLATION |73573 |0.011 |
PROSTITUTION |68635 |0.01 |
PUBLIC PEACE VIOLATION |48561 |0.007 |
OFFENSE INVOLVING CHILDREN |46697 |0.007 |
CRIM SEXUAL ASSAULT |28085 |0.004 |
SEX OFFENSE |25749 |0.004 |
INTERFERENCE WITH PUBLIC OFFICER |15801 |0.002 |
GAMBLING |14450 |0.002 |
LIQUOR LAW VIOLATION |14146 |0.002 |
ARSON |11336 |0.002 |
HOMICIDE |9680 |0.001 |
KIDNAPPING |6754 |0.001 |
INTIMIDATION |4028 |0.001 |
STALKING |3490 |0.001 |
OBSCENITY |607 |0.0 |
CONCEALED CARRY LICENSE VIOLATION|374 |0.0 |
PUBLIC INDECENCY |165 |0.0 |
HUMAN TRAFFICKING |57 |0.0 |
RITUALISM |23 |0.0 |
+---------------------------------+-------+-------+

[Chicago Police Department - Illinois Uniform Crime Reporting (IUCR) Codes](https://data.cityofchicago.org/Public-Safety/Chicago-Police-Department-Illinois-Uniform-Crime-R/c7ck-438e)

IUCR codes are four digit codes that law enforcement agencies use to classify criminal incidents when taking individual reports. These codes are also used to aggregate types of cases for statistical purposes. In Illinois, the Illinois State Police establish IUCR codes, but the agencies can add codes to suit their individual needs. The Chicago Police Department currently uses more than 350 IUCR codes to classify criminal offenses, divided into “Index” and “Non-Index” offenses. Index offenses are the offenses that are collected nation-wide by the Federal Bureaus of Investigation’s Uniform Crime Reports program to document crime trends over time (data released semi-annually), and include murder, criminal sexual assault, robbery, aggravated assault & battery, burglary, theft, motor vehicle theft, and arson. Non-index offenses are all other types of criminal incidents, including vandalism, weapons violations, public peace violations, etc.

Note: `iucr` will be used to define an Index/Non-Index offense variable.

In [46]:
from pyspark.sql.functions import countDistinct
chicago_crime.agg(countDistinct(chicago_crime.iucr)).show()

+--------------------+
count(DISTINCT iucr)|
+--------------------+
 396|
+--------------------+

In [47]:
chicago_crime.select('iucr').distinct().count()

Out[ 199 ]: 396

In [48]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'iucr', 396)

+----+------+-------+
iucr|count |percent|
+----+------+-------+
0820|557719|0.081 |
0486|528876|0.077 |
0460|510721|0.074 |
1320|368506|0.054 |
1310|361146|0.053 |
0810|355390|0.052 |
0560|297211|0.043 |
1811|277353|0.04 |
0610|265849|0.039 |
0910|250413|0.036 |
0890|232402|0.034 |
0860|172534|0.025 |
2820|136406|0.02 |
1330|121825|0.018 |
2027|119839|0.017 |
0620|104575|0.015 |
2825|101761|0.015 |
2024|93341 |0.014 |
031A|91894 |0.013 |
0320|89969 |0.013 |
0430|68748 |0.01 |
1150|55148 |0.008 |
143A|48878 |0.007 |
1506|47537 |0.007 |
051A|47305 |0.007 |
0840|44531 |0.006 |
041A|37074 |0.005 |
5002|36986 |0.005 |
1350|36501 |0.005 |
4387|34419 |0.005 |
0870|34276 |0.005 |
1305|34089 |0.005 |
1130|33447 |0.005 |
1152|32776 |0.005 |
1210|31688 |0.005 |
0520|30470 |0.004 |
0470|29270 |0.004 |
1153|27861 |0.004 |
0420|27332 |0.004 |
2092|27317 |0.004 |
0530|25522 |0.004 |
2014|25456 |0.004 |
1365|25453 |0.004 |
2826|25390 |0.004 |
2022|24994 |0.004 |
0915|24650 |0.004 |
2017|24539 |0.004 |
0930|22868 |0.003 |
2095|21313 |0.003 |
502P|21217 |0.003 |
0497|18795 |0.003 |
1812|18266 |0.003 |
1120|17568 |0.003 |
0630|17243 |0.003 |
2093|17022 |0.002 |
0880|16917 |0.002 |
0281|15714 |0.002 |
0484|15554 |0.002 |
0841|15206 |0.002 |
1513|14711 |0.002 |
0545|14675 |0.002 |
1110|14644 |0.002 |
1750|14098 |0.002 |
1345|13449 |0.002 |
0454|13448 |0.002 |
0330|13030 |0.002 |
0496|12924 |0.002 |
0313|12770 |0.002 |
1360|12457 |0.002 |
1121|12322 |0.002 |
1661|12139 |0.002 |
0312|12008 |0.002 |
5001|11659 |0.002 |
1822|11631 |0.002 |
1821|11601 |0.002 |
0920|11485 |0.002 |
0850|11382 |0.002 |
0340|10011 |0.001 |
0110|9634 |0.001 |
0326|9578 |0.001 |
1563|9273 |0.001 |
1780|9148 |0.001 |
2170|8890 |0.001 |
1220|8599 |0.001 |
141C|8523 |0.001 |
033A|8393 |0.001 |
1154|8242 |0.001 |
502R|7919 |0.001 |
1122|7783 |0.001 |
5007|7401 |0.001 |
1020|7388 |0.001 |
1570|7366 |0.001 |
5000|7136 |0.001 |
5011|7121 |0.001 |
1710|6821 |0.001 |
2850|6622 |0.001 |
0554|6390 |0.001 |
3710|5885 |0.001 |
1340|5797 |0.001 |
3731|5702 |0.001 |
1562|5693 |0.001 |
4625|5275 |0.001 |
0650|5190 |0.001 |
0325|5135 |0.001 |
0917|5080 |0.001 |
0440|5033 |0.001 |
1206|4876 |0.001 |
4651|4740 |0.001 |
2230|4641 |0.001 |
2851|4633 |0.001 |
2094|4619 |0.001 |
0265|4555 |0.001 |
2830|4126 |0.001 |
4650|4095 |0.001 |
2025|3941 |0.001 |
2091|3840 |0.001 |
0498|3838 |0.001 |
1790|3646 |0.001 |
141A|3619 |0.001 |
2026|3609 |0.001 |
2028|3554 |0.001 |
3960|3484 |0.001 |
1752|3410 |0.0 |
2023|3364 |0.0 |
501A|3291 |0.0 |
2220|3202 |0.0 |
2210|3133 |0.0 |
2250|3090 |0.0 |
1156|2989 |0.0 |
0843|2955 |0.0 |
1754|2929 |0.0 |
0580|2927 |0.0 |
0842|2901 |0.0 |
0266|2813 |0.0 |
0935|2809 |0.0 |
1200|2805 |0.0 |
143B|2779 |0.0 |
1477|2651 |0.0 |
2012|2625 |0.0 |
2900|2431 |0.0 |
1792|2421 |0.0 |
2860|2404 |0.0 |
0479|2384 |0.0 |
1505|2367 |0.0 |
0495|2331 |0.0 |
3730|2243 |0.0 |
0334|2192 |0.0 |
1512|2164 |0.0 |
1751|2151 |0.0 |
4255|2085 |0.0 |
4510|2056 |0.0 |
1460|2047 |0.0 |
051B|2024 |0.0 |
0337|2013 |0.0 |
1195|1986 |0.0 |
1025|1882 |0.0 |
1242|1852 |0.0 |
1090|1847 |0.0 |
2890|1753 |0.0 |
0261|1712 |0.0 |
2021|1622 |0.0 |
0453|1583 |0.0 |
2090|1566 |0.0 |
0499|1546 |0.0 |
143C|1526 |0.0 |
1753|1499 |0.0 |
3100|1468 |0.0 |
1140|1435 |0.0 |
1235|1400 |0.0 |
0483|1385 |0.0 |
031B|1343 |0.0 |
041B|1324 |0.0 |
0291|1272 |0.0 |
5111|1265 |0.0 |
0553|1229 |0.0 |
2840|1160 |0.0 |
0925|1154 |0.0 |
0558|1112 |0.0 |
0895|1107 |0.0 |
4230|1098 |0.0 |
1582|1069 |0.0 |
0263|1054 |0.0 |
2870|1052 |0.0 |
0550|985 |0.0 |
2111|969 |0.0 |
0331|956 |0.0 |
1170|932 |0.0 |
2110|916 |0.0 |
0865|911 |0.0 |
1151|866 |0.0 |
5004|863 |0.0 |
1755|852 |0.0 |
5110|837 |0.0 |
1185|826 |0.0 |
3800|823 |0.0 |
1375|818 |0.0 |
1525|812 |0.0 |
4310|787 |0.0 |
1670|726 |0.0 |
1544|709 |0.0 |
2013|702 |0.0 |
0485|697 |0.0 |
1155|672 |0.0 |
2020|667 |0.0 |
4210|658 |0.0 |
1585|657 |0.0 |
2015|657 |0.0 |
4388|651 |0.0 |
2050|651 |0.0 |
1565|650 |0.0 |
2031|643 |0.0 |
141B|608 |0.0 |
1720|595 |0.0 |
1680|576 |0.0 |
0552|546 

#### Create indicators for Index/Non-Index offense based on CSV file: [Chicago Police Department - Illinois Uniform Crime Reporting (IUCR) Codes](https://data.cityofchicago.org/Public-Safety/Chicago-Police-Department-Illinois-Uniform-Crime-R/c7ck-438e).

In [50]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('index_offense', when(chicago_crime.iucr.isin(['0110', '0130', '0261', '0262', '0263', '0264',
                                                                                        '0265', '0266', '0271', '0272', '0273', '0274',
                                                                                        '0275', '0281', '0291', '0312', '0313', '031A',
                                                                                        '031B', '0320', '0325', '0326', '0330', '0331',
                                                                                        '0334', '0337', '033A', '033B', '0340', '041A',
                                                                                        '041B', '0420', '0430', '0450', '0451', '0452',
                                                                                        '0453', '0461', '0462', '0479', '0480', '0481',
                                                                                        '0482', '0483', '0485', '0487', '0488', '0489',
                                                                                        '0490', '0491', '0492', '0493', '0495', '0496',
                                                                                        '0497', '0498', '0510', '051A', '051B', '0520',
                                                                                        '0530', '0550', '0551', '0552', '0553', '0555',
                                                                                        '0556', '0557', '0558', '0610', '0620', '0630',
                                                                                        '0650', '0810', '0820', '0850', '0860', '0865',
                                                                                        '0870', '0880', '0890', '0895', '0910', '0915',
                                                                                        '0917', '0918', '0920', '0925', '0927', '0928',
                                                                                        '0930', '0935', '0937', '0938', '1010', '1020',
                                                                                        '1025', '1090', '1753', '1754']), 1).otherwise(0))

In [51]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'index_offense')

+-------------+-------+-------+
index_offense|count |percent|
+-------------+-------+-------+
0 |4177733|0.608 |
1 |2698404|0.392 |
+-------------+-------+-------+

In [52]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('nonindex_offense', when(chicago_crime.iucr.isin(['0141', '0142', '0440', '0454', '0460', '0470',
                                                                                           '0475', '0484', '0486', '0494', '0545', '0554',
                                                                                           '0560', '0580', '0581', '0583', '0584', '1030',
                                                                                           '1035', '1050', '1055', '1110', '1120', '1121',
                                                                                           '1122', '1130', '1135', '1140', '1150', '1151',
                                                                                           '1152', '1153', '1154', '1155', '1156', '1160',
                                                                                           '1170', '1185', '1195', '1200', '1205', '1206',
                                                                                           '1210', '1220', '1230', '1235', '1240', '1241',
                                                                                           '1242', '1245', '1255', '1260', '1624', '1261',
                                                                                           '1265', '1305', '1310', '1320', '1330', '1335',
                                                                                           '1340', '1345', '1350', '1360', '1365', '1370',
                                                                                           '1375', '141A', '141B', '141C', '142A', '142B',
                                                                                           '1435', '143A', '143B', '143C', '1440', '1450',
                                                                                           '1460', '1475', '1476', '1477', '1478', '1479',
                                                                                           '1480', '1481', '1505', '1506', '1507', '1510',
                                                                                           '1511', '1512', '1513', '1515', '1520', '1521',
                                                                                           '1525', '1526', '1530', '1531', '1535', '1536',
                                                                                           '1537', '1540', '1541', '1542', '1544', '1549',
                                                                                           '1562', '1563', '1564', '1565', '1566', '1570',
                                                                                           '1572', '1574', '1576', '1578', '1580', '1582',
                                                                                           '1585', '1590', '1610', '1611', '1620', '1621',
                                                                                           '1622', '1623', '1625', '1626', '1627', '1630',
                                                                                           '1631', '1632', '1633', '1640', '1650', '1651',
                                                                                           '1661', '1670', '1680', '1681', '1682', '1690',
                                                                                           '1691', '1692', '1693', '1694', '1695', '1696',
                                                                                           '1697', '1710', '1715', '1720', '1725', '1750',
                                                                                           '1751', '1752', '1755', '1775', '1780', '1790',
                                                                                           '1791', '1792', '1811', '1812', '1821', '1822',
                                                                                           '1840', '1850', '1860', '1900', '2010', '2011',
                                                                                           '2012', '2013', '2014', '2015', '2016', '2017',
                                                                                           '2018', '2019', '2020', '2021', '2022', '2023',
                                                                                           '2024', '2025', '2026', '2027', '2028', '2029',
                                                                                           '2030', '2031', '2032', '2033', '2034', '2040',
                                                                                           '2050', '2060', '2070', '2080', '2090', '2091',
                                                                                           '2092', '2093', '2094', '2095', '2110', '2111',
                                                                                           '2120', '2160', '2170', '2210', '2220', '2230',
                                                                                           '2240', '2250', '2251', '2500', '2820', '2825',
                                                                                           '2826', '2830', '2840', '2850', '2851', '2860',
                                                                                           '2870', '2890', '2895', '2900', '3000', '3100',
                                                                                           '3200', '3300', '3400', '3610', '3710', '3720',
                                                                                           '3730', '3731', '3740', '3750', '3751', '3760',
                                                                                           '3770', '3800', '3910', '3920', '3960', '3966',
                                                                                           '3970', '3975', '3980', '4210', '4220', '4230',
                                                                                           '4240', '4255', '4310', '4386', '4387', '4388',
                                                                                           '4389', '4410', '4420', '4510', '4625', '4650',
                                                                                           '4651', '4652', '4740', '4750', '4800', '4810',
                                                                                           '4860', '5000', '5001', '5002', '5003', '5004',
                                                                                           '5007', '5009', '500E', '500N', '5011', '501A',
                                                                                           '501H', '502P', '502R', '502T', '5110', '5111',
                                                                                           '5112', '5120', '5121', '5122', '5130', '5131',
                                                                                           '5132']), 1).otherwise(0))

In [53]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'nonindex_offense')

+----------------+-------+-------+
nonindex_offense|count |percent|
+----------------+-------+-------+
1 |4110316|0.598 |
0 |2765821|0.402 |
+----------------+-------+-------+

In [54]:
# Compute a frequency table of two factors
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
#orange = sns.light_palette('orange', as_cmap=True)
#crosstab(chicago_crime.index_offense.fillna(' NaN'), chicago_crime.nonindex_offense.fillna(' NaN'), margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

In [55]:
# Computes a pair-wise frequency table of the given columns; also known as a contingency table
#chicago_crime.crosstab('index_offense', 'nonindex_offense').show()
display(chicago_crime.crosstab('index_offense', 'nonindex_offense'))

index_offense_nonindex_offense,0,1
1,2698404,0
0,67417,4110316


Note: There are 67417 records that are not consistent with the Index/Non-Index offense indicators as defined.

The following `iucr` codes are not contained in the CSV files as described above.

In [58]:
value_counts(chicago_crime.where((chicago_crime.index_offense == 0) & (chicago_crime.nonindex_offense == 0)), 'iucr')

+----+-----+-------+
iucr|count|percent|
+----+-----+-------+
0840|44531|0.661 |
0841|15206|0.226 |
0843|2955 |0.044 |
0842|2901 |0.043 |
0499|1546 |0.023 |
5008|189 |0.003 |
1581|72 |0.001 |
5005|6 |0.0 |
5013|5 |0.0 |
3961|4 |0.0 |
0830|1 |0.0 |
9901|1 |0.0 |
+----+-----+-------+

In [59]:
# Return DataFrame with duplicate rows removed, optionally only considering certain columns
chicago_crime.select(['iucr', 'index_offense', 'nonindex_offense', 'primary_type', 'description']).where((chicago_crime.index_offense == 0) & (chicago_crime.nonindex_offense == 0)).dropDuplicates().sort('iucr', ascending=True).show(truncate=False)

+----+-------------+----------------+-------------+--------------------------------------------------+
iucr|index_offense|nonindex_offense|primary_type |description |
+----+-------------+----------------+-------------+--------------------------------------------------+
0499|0 |0 |BATTERY |AGGRAVATED DOMESTIC BATTERY |
0830|0 |0 |THEFT |THEFT RETAIL |
0840|0 |0 |THEFT |FINANCIAL ID THEFT: OVER $300 |
0841|0 |0 |THEFT |FINANCIAL ID THEFT:$300 &UNDER |
0842|0 |0 |THEFT |AGG: FINANCIAL ID THEFT |
0843|0 |0 |THEFT |ATTEMPT FINANCIAL IDENTITY THEFT |
1581|0 |0 |SEX OFFENSE |NON-CONSENSUAL DISSEMINATION PRIVATE SEXUAL IMAGES|
3961|0 |0 |INTIMIDATION |INTIMIDATION OF LAW ENFORCEMENT OFFICIAL |
5005|0 |0 |SEX OFFENSE |ATT AGG CRIM SEXUAL ABUSE |
5008|0 |0 |OTHER OFFENSE|FIREARM REGISTRATION VIOLATION |
5013|0 |0 |OTHER OFFENSE|VIOLATION OF SUMMARY CLOSURE |
9901|0 |0 |ASSAULT |DOMESTIC VIOLENCE |
+----+-------------+----------------+-------------+--------------------------------------------------+

The [Chicago Police Department CLEARMAP Crime Summary](http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html#N04A) webpage was referenced to see whether the above IUCR codes could be categorized into Index/Non-Index indicators.

In [61]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('iucr_index_offense_update', when(chicago_crime.iucr.isin(['0499', '0830', '0840', '0841', '0842', '0843', '9901']), 1).otherwise(0))

In [62]:
value_counts(chicago_crime, 'iucr_index_offense_update')

+-------------------------+-------+-------+
iucr_index_offense_update|count |percent|
+-------------------------+-------+-------+
0 |6808996|0.99 |
1 |67141 |0.01 |
+-------------------------+-------+-------+

In [63]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('iucr_nonindex_offense_update', when(chicago_crime.iucr.isin(['1581', '3961', '5005', '5008', '5013']), 1).otherwise(0))

In [64]:
value_counts(chicago_crime, 'iucr_nonindex_offense_update')

+----------------------------+-------+-------+
iucr_nonindex_offense_update|count |percent|
+----------------------------+-------+-------+
0 |6875861|1.0 |
1 |276 |0.0 |
+----------------------------+-------+-------+

`index_offense` values will be updated for records that were flagged by `iucr_index_offense_update`.

In [66]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('index_offense', when(chicago_crime.iucr_index_offense_update == 1, 1).otherwise(chicago_crime.index_offense))
chicago_crime = chicago_crime.drop('iucr_index_offense_update')

`nonindex_offense` values will be updated for records that were flagged by `iucr_nonindex_offense_update`.

In [68]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('nonindex_offense', when(chicago_crime.iucr_nonindex_offense_update == 1, 1).otherwise(chicago_crime.nonindex_offense))
chicago_crime = chicago_crime.drop('iucr_nonindex_offense_update')

In [69]:
# Compute a frequency table of two factors
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
#orange = sns.light_palette('orange', as_cmap=True)
#pd.crosstab(chicago_crime.index_offense.fillna(' NaN'), chicago_crime.nonindex_offense.fillna(' NaN'), margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

In [70]:
# Computes a pair-wise frequency table of the given columns; also known as a contingency table
#chicago_crime.crosstab('index_offense', 'nonindex_offense').show()
display(chicago_crime.crosstab('index_offense', 'nonindex_offense'))

index_offense_nonindex_offense,0,1
1,2765545,0
0,0,4110592


In [71]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'index_offense')

+-------------+-------+-------+
index_offense|count |percent|
+-------------+-------+-------+
0 |4110592|0.598 |
1 |2765545|0.402 |
+-------------+-------+-------+

In [72]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'nonindex_offense')

+----------------+-------+-------+
nonindex_offense|count |percent|
+----------------+-------+-------+
1 |4110592|0.598 |
0 |2765545|0.402 |
+----------------+-------+-------+

Note: `description` is a secondary description of the IUCR code, a subcategory of the primary description. `primary_type`, the primary description, will be used instead for exploring data.

In [74]:
chicago_crime.select('description').distinct().count()

Out[ 219 ]: 374

In [75]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'description', 374)

+-----------------------------------------------------------+------+-------+
description |count |percent|
+-----------------------------------------------------------+------+-------+
SIMPLE |810859|0.118 |
$500 AND UNDER |557704|0.081 |
DOMESTIC BATTERY SIMPLE |528876|0.077 |
TO VEHICLE |380963|0.055 |
TO PROPERTY |361146|0.053 |
OVER $500 |355377|0.052 |
POSS: CANNABIS 30GMS OR LESS |277353|0.04 |
FORCIBLE ENTRY |265849|0.039 |
AUTOMOBILE |250413|0.036 |
FROM BUILDING |232402|0.034 |
RETAIL THEFT |172534|0.025 |
TELEPHONE THREAT |136406|0.02 |
TO LAND |121825|0.018 |
POSS: CRACK |119839|0.017 |
UNLAWFUL ENTRY |104575|0.015 |
HARASSMENT BY TELEPHONE |101761|0.015 |
AGGRAVATED: OTHER DANG WEAPON |94577 |0.014 |
POSS: HEROIN(WHITE) |93341 |0.014 |
ARMED: HANDGUN |91894 |0.013 |
STRONGARM - NO WEAPON |89969 |0.013 |
AGGRAVATED: HANDGUN |86091 |0.013 |
AGGRAVATED:KNIFE/CUTTING INSTR |57802 |0.008 |
CREDIT CARD FRAUD |55148 |0.008 |
UNLAWFUL POSS OF HANDGUN |48878 |0.007 |
SOLICIT ON PUBLIC WAY |47537 |0.007 |
FINANCIAL ID THEFT: OVER $300 |44531 |0.006 |
OTHER VEHICLE OFFENSE |36986 |0.005 |
TO STATE SUP LAND |36501 |0.005 |
VIOLATE ORDER OF PROTECTION |34419 |0.005 |
POCKET-PICKING |34276 |0.005 |
CRIMINAL DEFACEMENT |34089 |0.005 |
FRAUD OR CONFIDENCE GAME |33447 |0.005 |
ILLEGAL USE CASH CARD |32776 |0.005 |
THEFT OF LABOR/SERVICES |31688 |0.005 |
PRO EMP HANDS NO/MIN INJURY |30229 |0.004 |
RECKLESS CONDUCT |29270 |0.004 |
FINANCIAL IDENTITY THEFT OVER $ 300 |27861 |0.004 |
SOLICIT NARCOTICS ON PUBLICWAY |27317 |0.004 |
MANU/DELIVER: HEROIN (WHITE) |25456 |0.004 |
TO RESIDENCE |25453 |0.004 |
HARASSMENT BY ELECTRONIC MEANS |25390 |0.004 |
POSS: COCAINE |24994 |0.004 |
TRUCK, BUS, MOTOR HOME |24650 |0.004 |
MANU/DELIVER:CRACK |24539 |0.004 |
THEFT/RECOVERY: AUTOMOBILE |22868 |0.003 |
ATTEMPT POSSESSION NARCOTICS |21313 |0.003 |
FALSE/STOLEN/ALTERED TRP |21217 |0.003 |
AGG PO HANDS NO/MIN INJURY |19838 |0.003 |
AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON |18795 |0.003 |
POSS: CANNABIS MORE THAN 30GMS |18266 |0.003 |
FORGERY |17568 |0.003 |
ATTEMPT FORCIBLE ENTRY |17243 |0.003 |
FOUND SUSPECT NARCOTICS |17022 |0.002 |
PURSE-SNATCHING |16917 |0.002 |
NON-AGGRAVATED |15714 |0.002 |
AGGRAVATED |15464 |0.002 |
FINANCIAL ID THEFT:$300 &UNDER |15206 |0.002 |
SOLICIT FOR BUSINESS |14711 |0.002 |
BOGUS CHECK |14644 |0.002 |
CHILD ABUSE |14098 |0.002 |
TO CITY OF CHICAGO PROPERTY |13449 |0.002 |
AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST |12924 |0.002 |
ARMED: OTHER DANGEROUS WEAPON |12770 |0.002 |
COUNTERFEITING DOCUMENT |12322 |0.002 |
GAME/DICE |12139 |0.002 |
ARMED:KNIFE/CUTTING INSTRUMENT |12008 |0.002 |
OTHER CRIME INVOLVING PROPERTY |11659 |0.002 |
MANU/DEL:CANNABIS OVER 10 GMS |11631 |0.002 |
MANU/DEL:CANNABIS 10GM OR LESS |11601 |0.002 |
ATT: AUTOMOBILE |11485 |0.002 |
ATTEMPT THEFT |11382 |0.002 |
ATTEMPT: STRONGARM-NO WEAPON |10011 |0.001 |
FIRST DEGREE MURDER |9634 |0.001 |
AGGRAVATED VEHICULAR HIJACKING |9578 |0.001 |
CRIMINAL SEXUAL ABUSE |9273 |0.001 |
OTHER OFFENSE |9148 |0.001 |
POSSESSION OF DRUG EQUIPMENT |8890 |0.001 |
THEFT OF LOST/MISLAID PROP |8599 |0.001 |
UNLAWFUL USE OTHER DANG WEAPON |8523 |0.001 |
ATTEMPT: ARMED-HANDGUN |8393 |0.001 |
FINANCIAL IDENTITY THEFT $300 AND UNDER |8242 |0.001 |
VEHICLE TITLE/REG OFFENSE |7919 |0.001 |
COUNTERFEIT CHECK |7783 |0.001 |
OTHER WEAPONS VIOLATION |7401 |0.001 |
BY FIRE |7388 |0.001 |
PUBLIC INDECENCY |7366 |0.001 |
OTHER CRIME AGAINST PERSON |7136 |0.001 |
LICENSE VIOLATION |7121 |0.001 |
ENDANGERING LIFE/HEALTH CHILD |6821 |0.001 |
BOMB THREAT |6622 |0.001 |
RESIST/OBSTRUCT/DISARM OFFICER |5885 |0.001 |
TO STATE SUP PROP |5797 |0.001 |
OBSTRUCTING IDENTIFICATION |5702 |0.001 |
AGG CRIMINAL SEXUAL ABUSE |5693 |0.001 |
PAROLE VIOLATION |5275 |0.001 |
HOME INVASION |5190 |0.001 |
VEHICULAR HIJACKING |5135 |0.001 |
CYCLE, SCOOTER, BIKE W-VIN |5080 |0.001 |
AGG: HANDS/FIST/FEET NO/MINOR INJURY |5033 |0.001 |
THEFT BY LESSEE,MOTOR VEH |4876 |0.001 |
SEX OFFENDER: FAIL RE

In [76]:
chicago_crime.select('location_description').distinct().count()

Out[ 221 ]: 180

In [77]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'location_description', 180)

+-----------------------------------------------------+-------+-------+
location_description |count |percent|
+-----------------------------------------------------+-------+-------+
STREET |1798094|0.261 |
RESIDENCE |1164770|0.169 |
APARTMENT |715533 |0.104 |
SIDEWALK |674237 |0.098 |
OTHER |261800 |0.038 |
PARKING LOT/GARAGE(NON.RESID.) |197412 |0.029 |
ALLEY |153088 |0.022 |
SCHOOL, PUBLIC, BUILDING |144339 |0.021 |
RESIDENCE-GARAGE |133048 |0.019 |
SMALL RETAIL STORE |122737 |0.018 |
RESIDENCE PORCH/HALLWAY |120341 |0.018 |
VEHICLE NON-COMMERCIAL |110651 |0.016 |
RESTAURANT |109257 |0.016 |
GROCERY FOOD STORE |89016 |0.013 |
DEPARTMENT STORE |86015 |0.013 |
GAS STATION |73797 |0.011 |
RESIDENTIAL YARD (FRONT/BACK) |71448 |0.01 |
CHA PARKING LOT/GROUNDS |55664 |0.008 |
PARK PROPERTY |52737 |0.008 |
COMMERCIAL / BUSINESS OFFICE |50007 |0.007 |
BAR OR TAVERN |37162 |0.005 |
CTA PLATFORM |36876 |0.005 |
CHA APARTMENT |36344 |0.005 |
DRUG STORE |31119 |0.005 |
SCHOOL, PUBLIC, GROUNDS |29571 |0.004 |
HOTEL/MOTEL |28758 |0.004 |
BANK |27981 |0.004 |
CTA TRAIN |25185 |0.004 |
CHA HALLWAY/STAIRWELL/ELEVATOR |24903 |0.004 |
VACANT LOT/LAND |24191 |0.004 |
TAVERN/LIQUOR STORE |22063 |0.003 |
CTA BUS |22003 |0.003 |
HOSPITAL BUILDING/GROUNDS |21217 |0.003 |
DRIVEWAY - RESIDENTIAL |19898 |0.003 |
POLICE FACILITY/VEH PARKING LOT |17914 |0.003 |
CONVENIENCE STORE |17095 |0.002 |
AIRPORT/AIRCRAFT |16108 |0.002 |
CHURCH/SYNAGOGUE/PLACE OF WORSHIP |15144 |0.002 |
GOVERNMENT BUILDING/PROPERTY |14362 |0.002 |
NURSING HOME/RETIREMENT HOME |13927 |0.002 |
SCHOOL, PRIVATE, BUILDING |13773 |0.002 |
CONSTRUCTION SITE |13062 |0.002 |
ABANDONED BUILDING |11283 |0.002 |
CURRENCY EXCHANGE |10890 |0.002 |
CTA GARAGE / OTHER PROPERTY |10075 |0.001 |
WAREHOUSE |9362 |0.001 |
ATHLETIC CLUB |8283 |0.001 |
BARBERSHOP |7819 |0.001 |
ATM (AUTOMATIC TELLER MACHINE) |7721 |0.001 |
TAXICAB |7347 |0.001 |
MEDICAL/DENTAL OFFICE |7244 |0.001 |
FACTORY/MANUFACTURING BUILDING |6819 |0.001 |
CTA BUS STOP |6358 |0.001 |
LIBRARY |6047 |0.001 |
OTHER RAILROAD PROP / TRAIN DEPOT |5835 |0.001 |
COLLEGE/UNIVERSITY GROUNDS |5690 |0.001 |
VEHICLE-COMMERCIAL |5483 |0.001 |
SPORTS ARENA/STADIUM |5122 |0.001 |
null |5013 |0.001 |
CLEANING STORE |4775 |0.001 |
CTA STATION |4462 |0.001 |
SCHOOL, PRIVATE, GROUNDS |4171 |0.001 |
AIRPORT TERMINAL UPPER LEVEL - SECURE AREA |3937 |0.001 |
OTHER COMMERCIAL TRANSPORTATION |2988 |0.0 |
DAY CARE CENTER |2845 |0.0 |
CAR WASH |2836 |0.0 |
MOVIE HOUSE/THEATER |2633 |0.0 |
APPLIANCE STORE |1835 |0.0 |
AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA |1669 |0.0 |
COLLEGE/UNIVERSITY RESIDENCE HALL |1381 |0.0 |
AUTO |1133 |0.0 |
LAKEFRONT/WATERFRONT/RIVERBANK |1129 |0.0 |
JAIL / LOCK-UP FACILITY |1063 |0.0 |
COIN OPERATED MACHINE |1053 |0.0 |
HIGHWAY/EXPRESSWAY |1045 |0.0 |
FIRE STATION |1023 |0.0 |
DELIVERY TRUCK |962 |0.0 |
POOL ROOM |899 |0.0 |
AIRPORT VENDING ESTABLISHMENT |894 |0.0 |
AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA |842 |0.0 |
FEDERAL BUILDING |796 |0.0 |
ANIMAL HOSPITAL |785 |0.0 |
AIRPORT EXTERIOR - NON-SECURE AREA |775 |0.0 |
AIRPORT PARKING LOT |714 |0.0 |
AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA |697 |0.0 |
BOWLING ALLEY |684 |0.0 |
BOAT/WATERCRAFT |668 |0.0 |
AIRCRAFT |646 |0.0 |
AIRPORT TERMINAL LOWER LEVEL - SECURE AREA |630 |0.0 |
AIRPORT BUILDING NON-TERMINAL - SECURE AREA |569 |0.0 |
PAWN SHOP |555 |0.0 |
HOUSE |536 |0.0 |
CREDIT UNION |502 |0.0 |
BRIDGE |389 |0.0 |
FOREST PRESERVE |382 |0.0 |
SAVINGS AND LOAN |358 |0.0 |
CEMETARY |354 |0.0 |
VEHICLE - OTHER RIDE SERVICE |331 |0.0 |
AUTO / BOAT / RV DEALERSHIP |317 |0.0 |
VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)|306 |0.0 |
PORCH |294 |0.0 |
AIRPORT EXTERIOR - SECURE AREA |281 |0.0 |
NEWSSTAND |233 |0.0 |
YARD |210 |0.0 |
PARKING LOT |161 |0.0 |
VEHICLE - DELIVERY TRUCK |131 |0.0 |
CTA TRACKS - RIGHT OF WAY |123 |0.0 |
VACANT LOT |103 |0.0 |
AIRPORT TRANSPORTATION SYSTEM (ATS) |90 |0.0 |
HALLWAY |84 |0.0 |
AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA

#### `location_description` values that were less than 1% of the distribution will be combined into the 'OTHER' group and a new variable will be creatd as `location_description_cat`.

In [79]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
gte_1pct = ['STREET', 'RESIDENCE', 'APARTMENT', 'SIDEWALK', 'OTHER', 'PARKING LOT/GARAGE(NON.RESID.)', 'ALLEY',
            'SCHOOL, PUBLIC, BUILDING', 'RESIDENCE-GARAGE', 'SMALL RETAIL STORE', 'RESIDENCE PORCH/HALLWAY',
            'VEHICLE NON-COMMERCIAL', 'RESTAURANT', 'GROCERY FOOD STORE', 'DEPARTMENT STORE', 'GAS STATION',
            'RESIDENTIAL YARD (FRONT/BACK)']
chicago_crime = chicago_crime.withColumn('location_description_gte_1pct', when(chicago_crime.location_description.isin(gte_1pct), 1).otherwise(0))

In [80]:
value_counts(chicago_crime, 'location_description_gte_1pct')

+-----------------------------+-------+-------+
location_description_gte_1pct|count |percent|
+-----------------------------+-------+-------+
1 |6025583|0.876 |
0 |850554 |0.124 |
+-----------------------------+-------+-------+

In [81]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('location_description_cat', when(chicago_crime.location_description_gte_1pct == 0, 'OTHER').otherwise(chicago_crime.location_description))
chicago_crime = chicago_crime.drop('location_description_gte_1pct')

In [82]:
value_counts(chicago_crime, 'location_description_cat')

+------------------------------+-------+-------+
location_description_cat |count |percent|
+------------------------------+-------+-------+
STREET |1798094|0.261 |
RESIDENCE |1164770|0.169 |
OTHER |1112354|0.162 |
APARTMENT |715533 |0.104 |
SIDEWALK |674237 |0.098 |
PARKING LOT/GARAGE(NON.RESID.)|197412 |0.029 |
ALLEY |153088 |0.022 |
SCHOOL, PUBLIC, BUILDING |144339 |0.021 |
RESIDENCE-GARAGE |133048 |0.019 |
SMALL RETAIL STORE |122737 |0.018 |
RESIDENCE PORCH/HALLWAY |120341 |0.018 |
VEHICLE NON-COMMERCIAL |110651 |0.016 |
RESTAURANT |109257 |0.016 |
GROCERY FOOD STORE |89016 |0.013 |
DEPARTMENT STORE |86015 |0.013 |
GAS STATION |73797 |0.011 |
RESIDENTIAL YARD (FRONT/BACK) |71448 |0.01 |
+------------------------------+-------+-------+

#### Create indicator (1/0) for whether an arrest was made.

In [84]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'arrest')

+------+-------+-------+
arrest|count |percent|
+------+-------+-------+
false |4976643|0.724 |
true |1899494|0.276 |
+------+-------+-------+

In [85]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('arrest', when(chicago_crime.arrest == False, 0).otherwise(1))

In [86]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'arrest')

+------+-------+-------+
arrest|count |percent|
+------+-------+-------+
0 |4976643|0.724 |
1 |1899494|0.276 |
+------+-------+-------+

#### Create indicator (1/0) for whether the incident was domestic-related.

In [88]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'domestic')

+--------+-------+-------+
domestic|count |percent|
+--------+-------+-------+
false |5969459|0.868 |
true |906678 |0.132 |
+--------+-------+-------+

In [89]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('domestic', when(chicago_crime.domestic == False, 0).otherwise(1))

In [90]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'domestic')

+--------+-------+-------+
domestic|count |percent|
+--------+-------+-------+
0 |5969459|0.868 |
1 |906678 |0.132 |
+--------+-------+-------+

In [91]:
chicago_crime.select('beat').distinct().count()

Out[ 233 ]: 304

In [92]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'beat', 304)

+----+-----+-------+
beat|count|percent|
+----+-----+-------+
423 |53538|0.008 |
421 |52756|0.008 |
624 |47655|0.007 |
1533|46488|0.007 |
511 |46424|0.007 |
1112|46175|0.007 |
823 |45835|0.007 |
1834|45188|0.007 |
414 |43428|0.006 |
1522|43356|0.006 |
2533|42465|0.006 |
621 |41767|0.006 |
612 |40943|0.006 |
321 |40504|0.006 |
825 |40443|0.006 |
522 |39354|0.006 |
713 |39198|0.006 |
512 |38976|0.006 |
1532|38326|0.006 |
631 |38118|0.006 |
1531|37894|0.006 |
1011|37408|0.005 |
1121|37113|0.005 |
1122|36987|0.005 |
831 |36968|0.005 |
712 |36449|0.005 |
122 |35965|0.005 |
832 |35873|0.005 |
312 |35755|0.005 |
632 |35468|0.005 |
324 |35410|0.005 |
623 |35130|0.005 |
513 |35066|0.005 |
1831|34955|0.005 |
833 |34934|0.005 |
2534|34587|0.005 |
813 |34458|0.005 |
1133|34103|0.005 |
1523|34084|0.005 |
1132|33919|0.005 |
523 |33918|0.005 |
611 |33488|0.005 |
725 |33280|0.005 |
834 |32477|0.005 |
2512|32328|0.005 |
835 |32117|0.005 |
323 |32034|0.005 |
424 |31947|0.005 |
824 |31927|0.005 |
132 |31750|0.005 |
815 |31670|0.005 |
412 |31657|0.005 |
313 |31602|0.005 |
524 |31227|0.005 |
331 |31145|0.005 |
1111|31002|0.005 |
2535|30951|0.005 |
2532|30871|0.004 |
1524|30850|0.004 |
332 |30534|0.004 |
1113|30530|0.004 |
2521|30390|0.004 |
912 |30378|0.004 |
613 |30301|0.004 |
533 |30094|0.004 |
1723|29980|0.004 |
1833|29966|0.004 |
711 |29834|0.004 |
2413|29565|0.004 |
1135|29465|0.004 |
723 |29358|0.004 |
2212|29345|0.004 |
431 |29309|0.004 |
733 |29046|0.004 |
413 |28903|0.004 |
735 |28734|0.004 |
1021|28703|0.004 |
2422|28702|0.004 |
2234|28631|0.004 |
1424|28392|0.004 |
2515|28357|0.004 |
1022|28117|0.004 |
422 |28026|0.004 |
432 |27943|0.004 |
411 |27877|0.004 |
1422|27806|0.004 |
1821|27747|0.004 |
1134|27721|0.004 |
2223|27522|0.004 |
1822|27466|0.004 |
822 |27301|0.004 |
111 |27251|0.004 |
322 |27103|0.004 |
2222|27054|0.004 |
1211|27005|0.004 |
634 |27001|0.004 |
1123|26827|0.004 |
914 |26794|0.004 |
123 |26777|0.004 |
1824|26641|0.004 |
2232|26582|0.004 |
1434|26152|0.004 |
614 |26108|0.004 |
1633|25900|0.004 |
531 |25870|0.004 |
1712|25702|0.004 |
1024|25479|0.004 |
2233|25457|0.004 |
1433|25328|0.004 |
1031|25294|0.004 |
726 |25257|0.004 |
934 |25118|0.004 |
1511|25071|0.004 |
2531|25034|0.004 |
1423|24996|0.004 |
1124|24937|0.004 |
1115|24911|0.004 |
2514|24826|0.004 |
334 |24647|0.004 |
1924|24625|0.004 |
915 |24464|0.004 |
1131|24225|0.004 |
112 |24089|0.004 |
1014|24051|0.003 |
211 |24003|0.003 |
913 |23982|0.003 |
2433|23945|0.003 |
532 |23925|0.003 |
2522|23874|0.003 |
1622|23857|0.003 |
1732|23788|0.003 |
1012|23771|0.003 |
131 |23753|0.003 |
1731|23690|0.003 |
1823|23662|0.003 |
2432|23623|0.003 |
814 |23619|0.003 |
1013|23602|0.003 |
1034|23581|0.003 |
1923|23437|0.003 |
924 |23326|0.003 |
935 |23277|0.003 |
932 |23251|0.003 |
1512|23159|0.003 |
1421|23155|0.003 |
212 |23126|0.003 |
731 |23114|0.003 |
1432|22816|0.003 |
724 |22788|0.003 |
1634|22729|0.003 |
1832|22673|0.003 |
1623|22585|0.003 |
2412|22423|0.003 |
2411|22392|0.003 |
734 |22220|0.003 |
1722|22189|0.003 |
2221|22169|0.003 |
333 |22166|0.003 |
922 |22095|0.003 |
1913|22063|0.003 |
714 |22010|0.003 |
732 |21983|0.003 |
1231|21861|0.003 |
722 |21664|0.003 |
1933|21635|0.003 |
2523|21632|0.003 |
233 |21592|0.003 |
911 |21389|0.003 |
2524|21256|0.003 |
1223|21217|0.003 |
622 |21188|0.003 |
811 |21115|0.003 |
222 |21018|0.003 |
1624|20976|0.003 |
2011|20957|0.003 |
314 |20879|0.003 |
433 |20849|0.003 |
1224|20827|0.003 |
821 |20758|0.003 |
1733|20662|0.003 |
2424|20661|0.003 |
1033|20462|0.003 |
113 |20382|0.003 |
1513|20318|0.003 |
1412|20190|0.003 |
715 |20158|0.003 |
923 |20134|0.003 |
1212|20098|0.003 |
232 |19969|0.003 |
931 |19836|0.003 |
633 |19578|0.003 |
2211|19352|0.003 |
234 |19313|0.003 |
1724|19171|0.003 |
1023|19112|0.003 |
2213|19080|0.003 |
1222|19076|0.003 |
1811|19027|0.003 |
1323|19014|0.003 |
1411|18993|0.003 |
933 |18975|0.003 |
1651|18552|0.003 |
1413|18549|0.003 |
1812|18503|0.003 |
1932|18474|0.003 |
1213|18200|0.003 |
2525|18196|0.003 |
223 |

Note: `beat` 430 and 310 have very low frequencies. After examining [Chicago Data Portal: Current police beat boundaries](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-current-/aerh-rz74), these don't appear to be active beat numbers.

#### Drop low frequency `beat` observations.

In [95]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('beat_low_freq', when(chicago_crime.beat.isin([430, 310]), 1).otherwise(0))

In [96]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'beat_low_freq')

+-------------+-------+-------+
beat_low_freq|count |percent|
+-------------+-------+-------+
0 |6876113|1.0 |
1 |24 |0.0 |
+-------------+-------+-------+

In [97]:
# Excluding low frequency values
chicago_crime = chicago_crime[chicago_crime.beat_low_freq == 0]
chicago_crime = chicago_crime.drop('beat_low_freq')

In [98]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6876113, Cols: 23

In [99]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'district')

+--------+------+-------+
district|count |percent|
+--------+------+-------+
8.0 |466921|0.068 |
11.0 |441052|0.064 |
7.0 |405315|0.059 |
6.0 |396763|0.058 |
25.0 |396111|0.058 |
4.0 |389519|0.057 |
3.0 |349128|0.051 |
9.0 |340932|0.05 |
12.0 |336816|0.049 |
2.0 |325980|0.047 |
19.0 |306529|0.045 |
5.0 |304781|0.044 |
18.0 |304275|0.044 |
15.0 |299500|0.044 |
10.0 |294750|0.043 |
14.0 |270683|0.039 |
1.0 |270064|0.039 |
16.0 |227604|0.033 |
22.0 |225216|0.033 |
24.0 |204938|0.03 |
17.0 |199157|0.029 |
20.0 |119841|0.017 |
31.0 |187 |0.0 |
null |47 |0.0 |
21.0 |4 |0.0 |
+--------+------+-------+

Can the 47 null values from `district` be filled in based on `beat` number?

Get length of `beat` column values.

In [102]:
chicago_crime = chicago_crime.withColumn('beat_length', F.length(chicago_crime.beat.astype('string')))

In [103]:
value_counts(chicago_crime, 'beat_length')

+-----------+-------+-------+
beat_length|count |percent|
+-----------+-------+-------+
4 |3739548|0.544 |
3 |3136565|0.456 |
+-----------+-------+-------+

Extract 1st number of `beat` if `beat_length` = 3; Extract 1st and 2nd number of `beat` if `beat_length` = 4.

In [105]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('district_update', when(chicago_crime.beat_length == 3, chicago_crime.beat.astype('string').substr(1, 1)).when(chicago_crime.beat_length == 4, chicago_crime.beat.astype('string').substr(1, 2)))

# Convert the column into type dataType
chicago_crime = chicago_crime.withColumn('district_update', chicago_crime.district_update.astype('double'))

In [106]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('district', when(chicago_crime.district.isNull(), chicago_crime.district_update).otherwise(chicago_crime.district))

In [107]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'district')

+--------+------+-------+
district|count |percent|
+--------+------+-------+
8.0 |466923|0.068 |
11.0 |441056|0.064 |
7.0 |405319|0.059 |
6.0 |396764|0.058 |
25.0 |396117|0.058 |
4.0 |389520|0.057 |
3.0 |349131|0.051 |
9.0 |340933|0.05 |
12.0 |336816|0.049 |
2.0 |325981|0.047 |
19.0 |306532|0.045 |
5.0 |304782|0.044 |
18.0 |304280|0.044 |
15.0 |299500|0.044 |
10.0 |294752|0.043 |
14.0 |270686|0.039 |
1.0 |270066|0.039 |
16.0 |227606|0.033 |
22.0 |225218|0.033 |
24.0 |204939|0.03 |
17.0 |199158|0.029 |
20.0 |119841|0.017 |
31.0 |187 |0.0 |
21.0 |4 |0.0 |
13.0 |2 |0.0 |
+--------+------+-------+

Note: `district`s 21 and 13 have very low frequencies. After examining [Chicago Data Portal: Current police district boundaries](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Districts-current-/fthy-xz3r), these don't appear to be active district numbers.

#### Drop inactive `district` observations

In [110]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('district_inactive', when(chicago_crime.district.isin([21.0, 13.0]), 1).otherwise(0))

In [111]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'district_inactive')

+-----------------+-------+-------+
district_inactive|count |percent|
+-----------------+-------+-------+
0 |6876107|1.0 |
1 |6 |0.0 |
+-----------------+-------+-------+

In [112]:
# Excluding low frequency values
chicago_crime = chicago_crime[chicago_crime.district_inactive == 0]
chicago_crime = chicago_crime.drop('district_inactive')

In [113]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6876107, Cols: 25

Note: `district` 31 has low frequency.  It does appear in the file above. However, it doesn't appear on the [Chicago Police Department: Districts](https://home.chicagopolice.org/community/districts/) webpage and isn't an active district.

In [115]:
chicago_crime.select('district').where(chicago_crime.district == 31.0).count()

Out[ 250 ]: 187

In [116]:
#chicago_crime.where(chicago_crime.district == 31.0).show(187, truncate=False)
display(chicago_crime.where(chicago_crime.district == 31.0))

date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,lat_long,index_offense,nonindex_offense,location_description_cat,beat_length,district_update
2012-05-21 14:05:00+00:00,064XX N MANNHEIM RD,2022,NARCOTICS,POSS: COCAINE,STREET,1,0,1654,31.0,null,0.0,18,1106267.0,1941351.0,2012,2018-02-09 15:44:29+00:00,41.99572241,-87.884462932,"(41.99572241, -87.884462932)",0,1,STREET,4,16.0
2009-01-28 13:26:00+00:00,097XX S AVENUE L,2012,NARCOTICS,MANU/DELIVER:COCAINE,VEHICLE NON-COMMERCIAL,1,0,432,31.0,10.0,52.0,18,1201792.0,1841217.0,2009,2018-02-28 15:56:25+00:00,41.719041877,-87.536494149,"(41.719041877, -87.536494149)",0,1,VEHICLE NON-COMMERCIAL,3,4.0
2009-02-17 17:00:00+00:00,099XX S EWING AVE,2012,NARCOTICS,MANU/DELIVER:COCAINE,VEHICLE NON-COMMERCIAL,1,0,432,31.0,10.0,52.0,18,1202137.0,1839849.0,2009,2016-02-04 06:33:39+00:00,41.71527921,-87.535277004,"(41.71527921, -87.535277004)",0,1,VEHICLE NON-COMMERCIAL,3,4.0
2008-05-13 20:35:00+00:00,052XX N HARLEM AVE,2015,NARCOTICS,MANU/DELIVER: HALLUCINOGEN,OTHER,1,0,1613,31.0,41.0,10.0,18,1127354.0,1934238.0,2008,2018-02-28 15:56:25+00:00,41.975874105,-87.807052779,"(41.975874105, -87.807052779)",0,1,OTHER,4,16.0
2018-03-27 16:30:00+00:00,064XX N MANNHEIM RD,0460,BATTERY,SIMPLE,STREET,0,0,1654,31.0,null,0.0,08B,1106267.0,1941351.0,2018,2018-05-04 15:51:04+00:00,41.99572241,-87.884462932,"(41.99572241, -87.884462932)",0,1,STREET,4,16.0
2013-12-19 13:35:00+00:00,065XX N MANNHEIM RD,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,0,0,1654,31.0,null,0.0,07,1106264.0,1941887.0,2013,2018-02-10 15:50:01+00:00,41.997193305,-87.884463632,"(41.997193305, -87.884463632)",1,0,STREET,4,16.0
2007-07-06 11:00:00+00:00,058XX N RIVER RD,0580,STALKING,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),0,0,1614,31.0,null,null,08A,1112570.0,1936925.0,2007,2018-10-26 16:01:05+00:00,41.98348393,-87.86136622,"(41.98348393, -87.86136622)",0,1,PARKING LOT/GARAGE(NON.RESID.),4,16.0
2008-05-04 13:00:00+00:00,058XX N RIVER RD,0820,THEFT,$500 AND UNDER,CTA GARAGE / OTHER PROPERTY,0,0,1614,31.0,null,null,06,1112570.0,1936925.0,2008,2018-10-26 16:01:05+00:00,41.98348393,-87.86136622,"(41.98348393, -87.86136622)",1,0,OTHER,4,16.0
2002-04-04 13:00:00+00:00,137XX S INDIANA AV,0820,THEFT,$500 AND UNDER,SIDEWALK,0,0,533,31.0,null,null,06,1179972.0,1814476.0,2002,2018-10-26 16:01:05+00:00,41.646187093,-87.61722683,"(41.646187093, -87.61722683)",1,0,SIDEWALK,3,5.0
2002-01-25 17:17:00+00:00,069XX N CENTRAL AV,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,0,0,1621,31.0,null,null,26,1139607.0,1945673.0,2002,2018-10-26 16:01:05+00:00,42.007037203,-87.761712432,"(42.007037203, -87.761712432)",0,1,RESIDENCE,4,16.0


#### Use `beat` to supply the `district` for inactive district 31.

In [118]:
# Replace values in a column
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name
# Evaluates a list of conditions and returns one of multiple possible result expressions; if Column.otherwise() is not invoked, None is returned for unmatched conditions
chicago_crime = chicago_crime.withColumn('district', when(chicago_crime.district == 31.0, chicago_crime.district_update).otherwise(chicago_crime.district))
chicago_crime = chicago_crime.drop('beat_length', 'district_update')

In [119]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'district')

+--------+------+-------+
district|count |percent|
+--------+------+-------+
8.0 |466929|0.068 |
11.0 |441061|0.064 |
7.0 |405321|0.059 |
6.0 |396764|0.058 |
25.0 |396125|0.058 |
4.0 |389535|0.057 |
3.0 |349143|0.051 |
9.0 |340934|0.05 |
12.0 |336816|0.049 |
2.0 |325981|0.047 |
19.0 |306533|0.045 |
5.0 |304785|0.044 |
18.0 |304280|0.044 |
15.0 |299503|0.044 |
10.0 |294753|0.043 |
14.0 |270687|0.039 |
1.0 |270067|0.039 |
16.0 |227687|0.033 |
22.0 |225237|0.033 |
24.0 |204966|0.03 |
17.0 |199158|0.029 |
20.0 |119842|0.017 |
+--------+------+-------+

In [120]:
chicago_crime.select('ward').distinct().count()

In [121]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'ward', 51)

+----+------+-------+
ward|count |percent|
+----+------+-------+
null|614818|0.089 |
28.0|286431|0.042 |
42.0|254297|0.037 |
24.0|246592|0.036 |
2.0 |240122|0.035 |
27.0|228668|0.033 |
17.0|208918|0.03 |
20.0|206557|0.03 |
6.0 |202647|0.029 |
3.0 |191472|0.028 |
21.0|188794|0.027 |
16.0|180032|0.026 |
34.0|176312|0.026 |
15.0|172425|0.025 |
8.0 |168570|0.025 |
37.0|168234|0.024 |
7.0 |166707|0.024 |
9.0 |158367|0.023 |
5.0 |149227|0.022 |
29.0|145029|0.021 |
1.0 |121235|0.018 |
4.0 |116841|0.017 |
26.0|107287|0.016 |
32.0|107177|0.016 |
10.0|107017|0.016 |
18.0|105304|0.015 |
49.0|88151 |0.013 |
30.0|87708 |0.013 |
31.0|87533 |0.013 |
44.0|86182 |0.013 |
25.0|86133 |0.013 |
11.0|85931 |0.012 |
14.0|85421 |0.012 |
46.0|83635 |0.012 |
35.0|83448 |0.012 |
23.0|81773 |0.012 |
43.0|79840 |0.012 |
13.0|79696 |0.012 |
12.0|76208 |0.011 |
22.0|72414 |0.011 |
38.0|66811 |0.01 |
36.0|65263 |0.009 |
41.0|64960 |0.009 |
47.0|64942 |0.009 |
40.0|64841 |0.009 |
50.0|63780 |0.009 |
45.0|63616 |0.009 |
33.0|62325 |0.009 |
48.0|62148 |0.009 |
39.0|58587 |0.009 |
19.0|55683 |0.008 |
+----+------+-------+

In [122]:
# Returns a new DataFrame containing the distinct values and sort
chicago_crime.select('ward').distinct().sort('ward', ascending=True).show(51, truncate=False)

+----+
ward|
+----+
null|
1.0 |
2.0 |
3.0 |
4.0 |
5.0 |
6.0 |
7.0 |
8.0 |
9.0 |
10.0|
11.0|
12.0|
13.0|
14.0|
15.0|
16.0|
17.0|
18.0|
19.0|
20.0|
21.0|
22.0|
23.0|
24.0|
25.0|
26.0|
27.0|
28.0|
29.0|
30.0|
31.0|
32.0|
33.0|
34.0|
35.0|
36.0|
37.0|
38.0|
39.0|
40.0|
41.0|
42.0|
43.0|
44.0|
45.0|
46.0|
47.0|
48.0|
49.0|
50.0|
+----+

In [123]:
chicago_crime.select('community_area').distinct().count()

Out[ 92 ]: 79

In [124]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'community_area', 79)

+--------------+------+-------+
community_area|count |percent|
+--------------+------+-------+
null |613491|0.089 |
25.0 |398032|0.058 |
8.0 |215160|0.031 |
43.0 |204476|0.03 |
23.0 |197935|0.029 |
28.0 |184858|0.027 |
24.0 |184853|0.027 |
67.0 |183831|0.027 |
29.0 |181238|0.026 |
71.0 |177379|0.026 |
49.0 |167107|0.024 |
68.0 |166923|0.024 |
66.0 |154948|0.023 |
69.0 |154218|0.022 |
32.0 |149768|0.022 |
44.0 |134624|0.02 |
22.0 |132075|0.019 |
61.0 |129356|0.019 |
6.0 |124191|0.018 |
27.0 |118733|0.017 |
26.0 |117078|0.017 |
46.0 |116565|0.017 |
19.0 |115573|0.017 |
30.0 |105554|0.015 |
53.0 |102770|0.015 |
42.0 |102402|0.015 |
7.0 |98208 |0.014 |
1.0 |95884 |0.014 |
3.0 |91557 |0.013 |
38.0 |86619 |0.013 |
15.0 |79748 |0.012 |
2.0 |78176 |0.011 |
73.0 |74097 |0.011 |
16.0 |71230 |0.01 |
35.0 |69844 |0.01 |
40.0 |67272 |0.01 |
77.0 |61582 |0.009 |
58.0 |61231 |0.009 |
31.0 |60928 |0.009 |
21.0 |58956 |0.009 |
63.0 |57823 |0.008 |
70.0 |57035 |0.008 |
14.0 |55936 |0.008 |
56.0 |52199 |0.008 |
75.0 |50146 |0.007 |
33.0 |47505 |0.007 |
65.0 |46874 |0.007 |
4.0 |43803 |0.006 |
51.0 |40582 |0.006 |
60.0 |40170 |0.006 |
41.0 |39024 |0.006 |
17.0 |38419 |0.006 |
20.0 |38005 |0.006 |
5.0 |37520 |0.005 |
76.0 |37315 |0.005 |
39.0 |35586 |0.005 |
48.0 |33857 |0.005 |
45.0 |32032 |0.005 |
52.0 |31194 |0.005 |
54.0 |27589 |0.004 |
10.0 |26654 |0.004 |
59.0 |25962 |0.004 |
50.0 |25150 |0.004 |
11.0 |24781 |0.004 |
64.0 |24747 |0.004 |
62.0 |24322 |0.004 |
34.0 |23227 |0.003 |
72.0 |22585 |0.003 |
57.0 |22571 |0.003 |
37.0 |21129 |0.003 |
13.0 |20920 |0.003 |
18.0 |14821 |0.002 |
74.0 |14047 |0.002 |
36.0 |13799 |0.002 |
55.0 |13557 |0.002 |
12.0 |11244 |0.002 |
47.0 |9363 |0.001 |
9.0 |6054 |0.001 |
0.0 |92 |0.0 |
+--------------+------+-------+

In [125]:
# Returns a new DataFrame containing the distinct values and sort
chicago_crime.select('community_area').distinct().sort('community_area', ascending=True).show(79, truncate=False)

+--------------+
community_area|
+--------------+
null |
0.0 |
1.0 |
2.0 |
3.0 |
4.0 |
5.0 |
6.0 |
7.0 |
8.0 |
9.0 |
10.0 |
11.0 |
12.0 |
13.0 |
14.0 |
15.0 |
16.0 |
17.0 |
18.0 |
19.0 |
20.0 |
21.0 |
22.0 |
23.0 |
24.0 |
25.0 |
26.0 |
27.0 |
28.0 |
29.0 |
30.0 |
31.0 |
32.0 |
33.0 |
34.0 |
35.0 |
36.0 |
37.0 |
38.0 |
39.0 |
40.0 |
41.0 |
42.0 |
43.0 |
44.0 |
45.0 |
46.0 |
47.0 |
48.0 |
49.0 |
50.0 |
51.0 |
52.0 |
53.0 |
54.0 |
55.0 |
56.0 |
57.0 |
58.0 |
59.0 |
60.0 |
61.0 |
62.0 |
63.0 |
64.0 |
65.0 |
66.0 |
67.0 |
68.0 |
69.0 |
70.0 |
71.0 |
72.0 |
73.0 |
74.0 |
75.0 |
76.0 |
77.0 |
+--------------+

Note: `community_area` 0 is not a valid community area. However, it doesn't apprear as though this value can be mapped to a valid value after exploring the data.

In [127]:
#chicago_crime.select(['iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic',
#                      'beat', 'district', 'ward', 'community_area', 'fbi_code']).where(chicago_crime.community_area == 0.0).show(92, truncate=False)
display(chicago_crime.select(['iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic',
                              'beat', 'district', 'ward', 'community_area', 'fbi_code']).where(chicago_crime.community_area == 0.0))

iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code
2022,NARCOTICS,POSS: COCAINE,STREET,1,0,1654,16,null,0.0,18
1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,BANK,0,0,1614,16.0,41.0,0.0,11
1206,DECEPTIVE PRACTICE,"THEFT BY LESSEE,MOTOR VEH",COMMERCIAL / BUSINESS OFFICE,0,0,1651,16.0,41.0,0.0,11
0460,BATTERY,SIMPLE,STREET,0,0,1654,16,null,0.0,08B
0460,BATTERY,SIMPLE,CTA PLATFORM,0,0,1614,16.0,41.0,0.0,08B
0460,BATTERY,SIMPLE,STREET,0,0,1614,16.0,41.0,0.0,08B
0460,BATTERY,SIMPLE,OTHER,0,0,124,1.0,42.0,0.0,08B
0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,0,1,1614,16.0,41.0,0.0,08B
0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,1,1,811,8.0,23.0,0.0,08B
0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,1,1,811,8.0,23.0,0.0,08B


Note: Both `ward` and `community_area` have 8.9% missing values.  It does not appear as though these values can be filled in from other information after exploring the data.  They will not be included in the modeling as these observations would be automatically dropped.

In [129]:
chicago_crime.select('fbi_code').distinct().count()

Out[ 96 ]: 26

In [130]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'fbi_code', 26)

+--------+-------+-------+
fbi_code|count |percent|
+--------+-------+-------+
06 |1448226|0.211 |
08B |1074131|0.156 |
14 |784261 |0.114 |
26 |699225 |0.102 |
18 |674890 |0.098 |
05 |392856 |0.057 |
08A |322820 |0.047 |
07 |318907 |0.046 |
03 |259579 |0.038 |
11 |230890 |0.034 |
04B |182379 |0.027 |
04A |110224 |0.016 |
15 |73947 |0.011 |
16 |68700 |0.01 |
24 |50273 |0.007 |
10 |37671 |0.005 |
02 |32513 |0.005 |
17 |32062 |0.005 |
20 |28526 |0.004 |
19 |14740 |0.002 |
22 |14146 |0.002 |
09 |11223 |0.002 |
01A |9635 |0.001 |
13 |2805 |0.0 |
12 |1435 |0.0 |
01B |45 |0.0 |
+--------+-------+-------+

In [131]:
# Compute a frequency table of two factors
# Builtin-Styles - https://pandas.pydata.org/pandas-docs/stable/style.html
#orange = sns.light_palette('orange', as_cmap=True)
#pd.crosstab(chicago_crime.iucr.fillna(' NaN'), chicago_crime.fbi_code.fillna(' NaN'), margins=True, margins_name='Total', dropna=False).style.background_gradient(cmap=orange)

In [132]:
# Computes a pair-wise frequency table of the given columns; also known as a contingency table
#chicago_crime.crosstab('iucr', 'fbi_code').show()
#display(chicago_crime.crosstab('iucr', 'fbi_code'))

Note: `iucr` codes are derived from the crime classification code (`fbi_code`).  Thus, `fbi_code` will not be used in the modeling.

In [134]:
# Categorical column frequency
# Returns counts of unique values in descending order (first element is the most frequently-occurring element)
# Note: Excludes NA values by default
value_counts(chicago_crime, 'year')

+----+------+-------+
year|count |percent|
+----+------+-------+
2002|486756|0.071 |
2001|485750|0.071 |
2003|475944|0.069 |
2004|469381|0.068 |
2005|453716|0.066 |
2006|448112|0.065 |
2007|437010|0.064 |
2008|427057|0.062 |
2009|392694|0.057 |
2010|370319|0.054 |
2011|351792|0.051 |
2012|335975|0.049 |
2013|307122|0.045 |
2014|275289|0.04 |
2016|268994|0.039 |
2017|268166|0.039 |
2018|266763|0.039 |
2015|264098|0.038 |
2019|91171 |0.013 |
+----+------+-------+

### Summarize the DataFrame

In [136]:
# Convert the column into type dataType
#chicago_crime = chicago_crime.withColumn('x_coordinate', chicago_crime.x_coordinate.astype('float'))
#chicago_crime = chicago_crime.withColumn('y_coordinate', chicago_crime.y_coordinate.astype('float'))
#chicago_crime = chicago_crime.withColumn('latitude', chicago_crime.latitude.astype('float'))
#chicago_crime = chicago_crime.withColumn('longitude', chicago_crime.longitude.astype('float'))

In [137]:
# Computes statistics for numeric columns
chicago_crime.describe('arrest', 'domestic', 'x_coordinate', 'y_coordinate', 'year', 'latitude', 'longitude', 'index_offense', 'nonindex_offense').show()

+-------+-------------------+-------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+
summary| arrest| domestic| x_coordinate| y_coordinate| year| latitude| longitude| index_offense| nonindex_offense|
+-------+-------------------+-------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+
 count| 6876107| 6876107| 6811139| 6811139| 6876107| 6811139| 6811139| 6876107| 6876107|
 mean|0.27624439235747783|0.13185891377199338|1164523.229881669|1885726.5563009358|2008.5136893594006| 41.84203024294263| -87.6717783920744| 0.40219473024489| 0.59780526975511|
 stddev| 0.4471391921147984|0.33833734242421215|17155.39213319576|32686.630906340375| 5.156358402297273|0.08994462917863105|0.06208631832696413|0.4903408652914224|0.49034086529142223|
 min| 0| 0| 0.0| 0.0| 2001| 36.619446| -91.68657| 0| 0|
 max| 1| 1| 1205119.0| 1951622.0| 2019| 42.02291| -87.52453| 1| 1|
+-------+-------------------+-------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+

Note: Exploring `x_coordinate` values.  The min value of 0.0 is quite far off from the rest of the values and may be an error.

In [139]:
#chicago_crime.select(['x_coordinate', 'y_coordinate', 'latitude', 'longitude']).where(chicago_crime.x_coordinate == 0.0)).show(truncate=False)
display(chicago_crime.select(['x_coordinate', 'y_coordinate', 'latitude', 'longitude']).where(chicago_crime.x_coordinate == 0.0))

x_coordinate,y_coordinate,latitude,longitude
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657
0.0,0.0,36.619446,-91.68657


In [140]:
chicago_crime.select(['x_coordinate', 'y_coordinate', 'latitude', 'longitude']).where(chicago_crime.x_coordinate == 0.0).count()

Out[ 138 ]: 161

In [141]:
# Computes statistics for numeric columns
chicago_crime.where(chicago_crime.x_coordinate != 0).describe('x_coordinate', 'y_coordinate', 'latitude', 'longitude').show()

+-------+------------------+------------------+-------------------+--------------------+
summary| x_coordinate| y_coordinate| latitude| longitude|
+-------+------------------+------------------+-------------------+--------------------+
 count| 6810978| 6810978| 6810978| 6810978|
 mean|1164550.7572411774|1885771.1316872556| 41.84215369600549| -87.6716834892099|
 stddev| 16194.36396122725|31374.864246831996|0.08628712634075986|0.058938763393995654|
 min| 1092706.0| 1813894.0| 41.644585| -87.934326|
 max| 1205119.0| 1951622.0| 42.02291| -87.52453|
+-------+------------------+------------------+-------------------+--------------------+

Note: These values appear to be much more realistic.  The 161 records will be dropped.

In [143]:
# Excluding unrealistic x_coordinate, y_coordinate, latitude, longitude values
chicago_crime = chicago_crime[chicago_crime.x_coordinate != 0.0]

In [144]:
# Return the number of rows and columns (dimensionality) of the DataFrame
print('Rows: {}, Cols: {}'.format(chicago_crime.count(), len(chicago_crime.columns)))

Rows: 6810978, Cols: 26

Note: Not possible to use PySpark describe function to compute statistics for object columns like in Python describe function.

In [146]:
# Describe object columns (e.g. categorical, strings, or timestamps)
# Generates descriptive summary statistics of the count, unique values, top values, and frequency
# Note: "top" is the most common value; "freq" is the most common value's frequency
#       Excludes "NaN" (missing) values
# Remove multiple columns
# Note: axis=0 for rows, 1 for columns
#round(chicago_crime.drop(['location_description', 'updated_on'], axis=1).describe(include='object'), 3)

In [147]:
from pyspark.mllib.stat import Statistics

In [148]:
# Calculate the correlation matrix using the default method (Pearson correlation)
col_names = chicago_crime.select(['x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'year']).columns
features = chicago_crime.select(['x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'year']).rdd.map(lambda row: row[0:])
corr_mat = Statistics.corr(features, method='pearson')
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names
#print(corr_df.to_string())
print(round(corr_df, 3))

x_coordinate y_coordinate latitude longitude year
x_coordinate 1.0 -0.558 -0.56 1.0 0.002
y_coordinate -0.558 1.0 1.0 -0.547 -0.003
latitude -0.56 1.0 1.0 -0.55 -0.003
longitude 1.0 -0.547 -0.55 1.0 0.002
year 0.002 -0.003 -0.003 0.002 1.0

In [149]:
# Calculate the correlation matrix using the default method (Pearson correlation)
#chicago_crime_correlations = chicago_crime[['x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'year']].corr(method='pearson')
#round(chicago_crime_correlations, 3)

In [150]:
# Create a figure and a set of subplots
# Note: figsize: a tuple (width, height) in inches
fig, ax = plt.subplots(figsize=(10, 8));

# Return an array of zeros with the same shape and type as a given array
mask = np.zeros_like(corr_df, dtype=np.bool)
# Return the indices for the upper-triangle of array
mask[np.triu_indices_from(mask)] = True

# Plot the correlation matrix (color-encoded) using Seaborn's .heatmap() function
# Note: vmin, vmax: values to anchor the colormap, otherwise they are inferred from the data & other keyword arguments
ax = sns.heatmap(corr_df, vmin=-1, vmax=1, mask=mask, ax=ax);

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14);
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14);

plt.show();

Note: `latitude` is highly correlated with `y_coordinate`; `longitude` is highly correlated with `x_coordinate`.  `latitude` and `longitude` are measures of location and are basically measuring the same thing as `x_coordinate` and `y_coordinate`.  Thus, `x_coordinate` and `y_coordinate` will not be used.

In [152]:
# For each index_offense group, describe latitude:
#round(chicago_crime.groupby(by='index_offense').latitude.describe(), 3)
chicago_crime.groupby('index_offense').agg(F.count('latitude').alias('count'), F.round(F.mean('latitude'), 3).alias('mean'), F.round(F.stddev('latitude'), 3).alias('stddev'), F.round(F.min('latitude'), 3).alias('min'), F.round(F.expr('percentile_approx(latitude, 0.25)'), 3).alias('25%'), F.round(F.expr('percentile_approx(latitude, 0.5)'), 3).alias('50%'), F.round(F.expr('percentile_approx(latitude, 0.75)'), 3).alias('75%'), F.round(F.max('latitude'), 3).alias('max')).orderBy('index_offense').show()

+-------------+-------+------+------+------+------+------+------+------+
index_offense| count| mean|stddev| min| 25%| 50%| 75%| max|
+-------------+-------+------+------+------+------+------+------+------+
 0|4071389|41.839| 0.086|41.645|41.767| 41.85|41.903|42.023|
 1|2739589|41.847| 0.086|41.645|41.772|41.866|41.912|42.023|
+-------------+-------+------+------+------+------+------+------+------+

[Independent Two-Sample T-test](https://pythonfordatascience.org/independent-t-test-python/)

In [154]:
# Calculate the T-test for the means of *two independent* samples of scores
# Note: nan_policy: defines how to handle when input contains nan. 'propagate' returns nan, 'raise' throws an error,
#                   'omit' performs the calculations ignoring nan values; default is 'propagate'
#statistic, pvalue = stats.ttest_ind(*chicago_crime.groupby(by='index_offense').latitude.apply(lambda x: list(x)), nan_policy='omit')

#print('t-statistic =', round(statistic, 3))
#print('p-value =', round(pvalue, 3))

In [155]:
#a = chicago_crime.groupby(by='index_offense').latitude.apply(lambda x: list(x))
#a0 = pd.Series(a[0])
#a1 = pd.Series(a[1])
#descriptives, results = rp.ttest(a0, a1, 'Non-Index', 'Index')
#round(descriptives, 3)

In [156]:
#round(results, 3)

In [157]:
# Create histograms
# Note: column: used to limit data to a subset of columns
#       by: used to form histograms for separate groups
#       sharex: share x axis and set some x axis labels to invisible
#       sharey: share y axis and set some y axis labels to invisible
#       layout: tuple of (rows, columns) for the layout of the histograms
#       figsize: a tuple (width, height) in inches
#chicago_crime.hist(column='latitude', by='index_offense', sharex=True, sharey=True, figsize=(10, 8));
#plt.xlabel('Latitude (Degrees)');
#plt.ylabel('Frequency');

In [158]:
# For each index_offense group, describe longitude:
#round(chicago_crime.groupby(by='index_offense').longitude.describe(), 3)
chicago_crime.groupby('index_offense').agg(F.count('longitude').alias('count'), F.round(F.mean('longitude'), 3).alias('mean'), F.round(F.stddev('longitude'), 3).alias('stddev'), F.round(F.min('longitude'), 3).alias('min'), F.round(F.expr('percentile_approx(longitude, 0.25)'), 3).alias('25%'), F.round(F.expr('percentile_approx(longitude, 0.5)'), 3).alias('50%'), F.round(F.expr('percentile_approx(longitude, 0.75)'), 3).alias('75%'), F.round(F.max('longitude'), 3).alias('max')).orderBy('index_offense').show()

+-------------+-------+-------+------+-------+-------+-------+-------+-------+
index_offense| count| mean|stddev| min| 25%| 50%| 75%| max|
+-------------+-------+-------+------+-------+-------+-------+-------+-------+
 0|4071390|-87.673| 0.06|-87.525|-87.717|-87.668|-87.629|-87.934|
 1|2739590| -87.67| 0.057|-87.525|-87.709|-87.664|-87.628|-87.934|
+-------------+-------+-------+------+-------+-------+-------+-------+-------+

In [159]:
# Calculate the T-test for the means of *two independent* samples of scores
# Note: nan_policy: defines how to handle when input contains nan. 'propagate' returns nan, 'raise' throws an error,
#                   'omit' performs the calculations ignoring nan values; default is 'propagate'
#statistic, pvalue = stats.ttest_ind(*chicago_crime.groupby(by='index_offense').longitude.apply(lambda x: list(x)), nan_policy='omit')

#print('t-statistic =', round(statistic, 3))
#print('p-value =', round(pvalue, 3))

In [160]:
#a = chicago_crime.groupby(by='index_offense').longitude.apply(lambda x: list(x))
#a0 = pd.Series(a[0])
#a1 = pd.Series(a[1])
#descriptives, results = rp.ttest(a0, a1, 'Non-Index', 'Index')
#round(descriptives, 3)

In [161]:
#round(results, 3)

In [162]:
# Create histograms
# Note: column: used to limit data to a subset of columns
#       by: used to form histograms for separate groups
#       sharex: share x axis and set some x axis labels to invisible
#       sharey: share y axis and set some y axis labels to invisible
#       layout: tuple of (rows, columns) for the layout of the histograms
#       figsize: a tuple (width, height) in inches
#chicago_crime.hist(column='longitude', by='index_offense', sharex=True, sharey=True, figsize=(10, 8));
#plt.xlabel('Longitude (Degrees)');
#plt.ylabel('Frequency');